# Project 2: Forest Fire Simulation

In this project, you will simulate the spread of a forest fire on a 100×100 grid. Each cell in the grid can be in one of four states: tree, grassland, burning tree, or burnt tree. The goal is to understand how forest density influences fire spread, and to explore extensions such as the effect of wind.

## Part 1: Initial Setup of the Grid
- Grid and States:
- Create a 100×100 NumPy array to represent the forest.
- Each cell can have one of the following values:
	0: Grassland (non-flammable)
	1: Tree
	2: Burning tree
	3: Burnt tree
- Random Initialization: 
	- Implement a function that initializes the grid with trees and grassland based on a tree density parameter (between 0 and 1).
	- Each cell should be a tree with probability = density, and grassland otherwise.
Lightning Strike:
	- Randomly select a cell that contains a tree (1) and set it to burning (2).

## Part 2: Fire Spread Simulation
- Simulation Step:
	- In each time step:
		- A burning tree (2) becomes a burnt tree (3).
		- All trees (1) that are directly adjacent (up, down, left, right) to a burning tree catch fire and become burning trees (2) in the next step.
- Run the Simulation:
	-Continue the simulation step-by-step until no more trees are burning.
- Track Results:
	- Record the number of trees burnt at the end of the simulation.
- Calculate the percentage of trees burnt compared to the initial number of trees.


## Part 3: Visualization and Analysis
- Graphical Representation:
	- Write a function that can visualize the grid at each time step by using matplotlib. Use different colors for each state.
	- Write a function that allows you to created an animated gif for a full run of the simulation, where each frame/picture corresponds to a time-step
- Density Curve:
	- Run the simulation for various density values (e.g., from 0.1 to 1.0 in steps of 0.05). Clearly you need to run the simulation several times for each density value and take then the mean value of the resulting percentage of trees burnt.
	- Plot the percentage of trees burnt as a function of the initial density including the statistical uncertainties.
- Critical Density:
	- From the plot, identify the critical density above which the fire spreads through almost the entire forest.
- Expert Challenge (not strictly required to be handed in, however is required for achieving the best mark): Larger grids, e.g. 1000x1000 require in a naive simulation significantly more time, hence it is advisable to think about certain optimization aspects. One promising approach is a clever usage of numpy arrays and slicing. You can add one additional "optimizedSimulation" function that can handle efficiently also larger map sizes and is more time-effective than a naive approach that directly loops on your arrays.

## Part 4: Extensions – Wind Effect
- Wind Influence:
	- Modify the fire-spread rule to account for wind blowing in one direction (e.g., east). For example, a tree that is east of a burning tree can catch fire even if it is not only the direct next neighbour. The strength of the wind might determine the spread radius in one direction. It is important that you come up here with your own model of how wind might effect the spread of fire. It is your task for explain the choice of your model. Think independently! As long as your model choice is sensible, you will get full points. 
Discuss:
	- What impact does wind have on the spread of the fire in your model?
	- How does the critical density change when wind is included?

## Deliverables:
- A Jupyter-Notebook implementing the simulation, including
	- A plot showing burnt tree percentage vs. initial density.
	- One Animation of the fire spreading at a given density and one lighting event.
	- A brief written summary of findings and observations about the critical density and wind effects.

## Task 1

In [109]:
# import needed modules
import numpy as np 
from matplotlib import pyplot as plt 


In [151]:
## legend
GRASSLAND = 0
TREE = 1
BURNING = 2
BURNT = 33

size = 3
forest = np.zeros([size,size])


def fill_forest(density):
    '''Creates an array of specified size with the probability for the trees being specified by the density. There is one burning tree due to a lightning strike'''

    # fill the forest with trees depending on the density
    forest[np.random.rand(size,size) < density] = 1

    # Select a random value from the forest array and set it to 2, to indicate a burning tree
    lightning_position = np.random.choice(size),np.random.choice(size)
    forest[lightning_position] = 2

fill_forest(0.7)


In [152]:
forest

array([[0., 1., 1.],
       [1., 1., 2.],
       [1., 1., 0.]])

In [153]:
def simulate_fire(forest):
    grid_size = forest.shape[0]
    history = [forest.copy()]
    
    while np.any(forest == BURNING):
        new_forest = forest.copy()

        for i in range(grid_size):
            for j in range(grid_size):
                if forest[i, j] == BURNING:
                    new_forest[i, j] = BURNT
                    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        ni, nj = i + dx, j + dy
                        if 0 <= ni < grid_size and 0 <= nj < grid_size:
                            if forest[ni, nj] == TREE:
                                new_forest[ni, nj] = BURNING

        forest = new_forest
        history.append(forest.copy())
    
    return history

In [149]:
simulate_fire(forest)


([array([[2., 1., 1.],
         [1., 1., 1.],
         [1., 0., 0.]]),
  array([[3., 2., 1.],
         [2., 1., 1.],
         [1., 0., 0.]]),
  array([[3., 3., 2.],
         [3., 2., 1.],
         [2., 0., 0.]]),
  array([[3., 3., 3.],
         [3., 3., 2.],
         [3., 0., 0.]]),
  array([[3., 3., 3.],
         [3., 3., 3.],
         [3., 0., 0.]])],
 array([[3., 3., 3.],
        [3., 3., 3.],
        [3., 0., 0.]]))